<a id='summary'></a>
# Spatialization
* Using the Orange OD Matrix as a transition matrix to draw destinations for the census equipped with agendas.


## Summary
* [Loading data](#data)
    * [GIS](#gis)
    * [transition_matrices](#od)
    * [synthetic pop](#synthpop) ([Grouping agendas](#Grouping_agendas))
    
    
* [Drawing](#drawing) 
    * [utilitary](#utilitary)
    * [parameters](#parameters)
    * [Parsing transition matrices into dictionnaries](#preproc)
    * [Functions to query probas](#proba)
    * [Running](#running)
    
    
* [Sanity checks](#sanity)
* [Adding empty agendas](#add_empty)
* [Exportation & conclusion](#export)


In [1]:
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import geopandas as gpd
import numba
from numba import njit, prange
from numba_progress import ProgressBar
from tqdm import tqdm

from utils.chrono import Chrono

with open('config.json', 'r') as config_path:
    config = json.load(config_path)

<a id='data'></a>
# Loading data

<a id='gis'></a>
## GIS
* Just to get the nearest neighbours for when agents ask for flows that do not exist in the transition matrix
* [back to summary](#summary)

In [2]:
gis_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['gis_map_filename'])
iris_commune = gpd.read_file(gis_path)

iris_commune = iris_commune.drop(columns=['wkt','geometry', 'is_iris'])

for col in iris_commune.columns:
    iris_commune[col] = iris_commune[col].astype(float)


<a id='od'></a>
## Transition matrices
* <font color='red'> **v8h**: Transition matrix is not differentiated by mode</font>
* [back to summary](#summary)

In [3]:
transition_matrix_path = os.path.join(config['outdata_dir']['path'],
                                      config['outdata_dir']['transition_matrix_by_dist_filename'])
c = Chrono('Loading transition matrix from {}'.format(transition_matrix_path))
trans_matrix = pd.read_csv(transition_matrix_path)
c.write('{} rows'.format(len(trans_matrix)))
trans_matrix.head()

15:21:52	Loading transition matrix from /Users/benoit/Desktop/Pro/210526-fusion/outdata/trans_matrix_by_distbin.csv
00:00:20	2274789 rows


,o,d,t,vol,proba_d,cart_key,x_o,y_o,geometry,x_d,y_d,centroid_dist,dist_bin_min,dist_bin_max
0,1043,1043,0.0,9.132230,1.0,1,3.932173e+06,2.538223e+06,"POLYGON ((3931567.736276 2535603.293955, 39314...",3.932173e+06,2.538223e+06,0.0,0,200.0
1,1043,1043,2.0,5.813602,1.0,1,3.932173e+06,2.538223e+06,"POLYGON ((3931567.736276 2535603.293955, 39314...",3.932173e+06,2.538223e+06,0.0,0,200.0
2,1043,1043,5.0,17.833213,1.0,1,3.932173e+06,2.538223e+06,"POLYGON ((3931567.736276 2535603.293955, 39314...",3.932173e+06,2.538223e+06,0.0,0,200.0
3,1043,1043,7.0,54.360548,1.0,1,3.932173e+06,2.538223e+06,"POLYGON ((3931567.736276 2535603.293955, 39314...",3.932173e+06,2.538223e+06,0.0,0,200.0
4,1043,1043,8.0,29.113536,1.0,1,3.932173e+06,2.538223e+06,"POLYGON ((3931567.736276 2535603.293955, 39314...",3.932173e+06,2.538223e+06,0.0,0,200.0


In [4]:
print('Number of parameters: {}'.format(len(trans_matrix)))
print('With dense matrices, could be: {}'.format(515*515*15))
nb_zeros = 515*515*15 - len(trans_matrix)
print('meaning we have {} zeros ({:.0%})'.format(nb_zeros, nb_zeros/(515*515*15)))

Number of parameters: 2274789
With dense matrices, could be: 3978375
meaning we have 1703586 zeros (43%)


<a id='synthpop'></a>
## Synthetic population
* [back to summary](#summary)

In [5]:
synthpop_path = os.path.join(config['outdata_dir']['path'], config['outdata_dir']['synthpop_rescaled_filename'])

c=Chrono('Loading agents from {}'.format(synthpop_path))
synthpop = pd.read_csv(synthpop_path)
c.tprint('{} rows, 1 row per agents'.format(len(synthpop)))

max_chain_len = synthpop['chain_len'].max()
c.tprint('max_chain_len = {}'.format(max_chain_len))

c.done()


15:22:13	Loading agents from /Users/benoit/Desktop/Pro/210526-fusion/outdata/synthpop/synthpop_statmatch_rescaleipu.csv
00:00:10	1335786 rows, 1 row per agents
00:00:10	max_chain_len = 10
00:00:10	Work complete !


<a id='Grouping_agendas'></a>
### Grouping agendas
* Agendas are considered equivalent if they share the same activity chain and the same home zone.
* Note that we could broaden the equivalence because the exact secondary purposes do not impact the spatialisation.
* But it doesn't change a lot actually.
* <font color='red'> **v8**: agendas that only differ in their transport modes are considered equivalent  </font>
* [back to summary](#summary)

In [6]:
c=Chrono('Grouping agendas...')
synthpop['id_agent'] = np.arange(len(synthpop))

agenda_cols = ['dep_0_motif'] + ['dep_{}_{}'.format(i, carac) for i in range(1,max_chain_len+1) 
               for carac in ['motif', 'time']]


synthpop_grouped = (synthpop[(synthpop['chain_len']>0) & (synthpop['autonomy']>0)]
                    .groupby(agenda_cols+['iris_or_commune'], dropna=False)
                    .agg({'chain_len':'first',
                          'gender':'count',
                          'autonomy':'first',
                          'id_agent':list})
                    .rename(columns={'gender':'vol'}).reset_index())

synthpop_empty_agenda = synthpop.loc[(synthpop['chain_len']==0) | (synthpop['autonomy']==0)].copy()

c.write('{} distinct agendas'.format(len(synthpop_grouped)))
c.write('{} agents with empty agendas, we don\'t spatialise them '
        'and they have to be added at the end'.format(len(synthpop_empty_agenda)))
c.done()


15:22:24	Grouping agendas...
00:00:11	390221 distinct agendas
00:00:11	283041 agents with empty agendas, we don't spatialise them and they have to be added at the end
00:00:11	Work complete !


## Counting number of meaningully different agendas
* All secondary activities are the same
* Primary activities that happen only once in the agenda are like a secondary activity

In [7]:
def get_equiv_mask(purpose_row):
    """
    input : a list of activity purposes
    return : primary activities happening only one are counted as secondary. 
            secondary activities are all counted as 3
    """
    res = purpose_row.copy()
    for p in [0,1,2]:
        if (purpose_row==p).sum()<=1:
            purpose_row[purpose_row==p] = 3
    purpose_row[purpose_row>=3] = 3
    return purpose_row

get_equiv_mask(np.array([0,1,0,2,3,0]))

array([0, 3, 0, 3, 3, 0])

In [8]:

sgr = synthpop_grouped.groupby(agenda_cols, dropna=False).size().reset_index()

purposes = sgr[['dep_0_motif'] + ['dep_{}_motif'.format(i) for i in range(1,max_chain_len+1)]].values
for i, row in enumerate(purposes):
    purposes[i] = get_equiv_mask(row)

sgr[['dep_0_motif'] + ['dep_{}_motif'.format(i) for i in range(1,max_chain_len+1)]] = purposes
sgr.groupby(agenda_cols, dropna=False).size().reset_index()

,dep_0_motif,dep_1_motif,dep_1_time,dep_2_motif,dep_2_time,dep_3_motif,dep_3_time,dep_4_motif,dep_4_time,dep_5_motif,...,dep_6_time,dep_7_motif,dep_7_time,dep_8_motif,dep_8_time,dep_9_motif,dep_9_time,dep_10_motif,dep_10_time,0
0,0.0,1.0,2.0,0.0,9.0,1.0,14.0,0.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0,1.0,2.0,0.0,10.0,1.0,12.0,0.0,16.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.0,1.0,2.0,0.0,10.0,1.0,12.0,0.0,19.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.0,1.0,2.0,0.0,10.0,1.0,14.0,0.0,19.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.0,1.0,2.0,0.0,12.0,1.0,16.0,0.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812,3.0,3.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2813,3.0,3.0,20.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2814,3.0,3.0,20.0,3.0,22.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2815,3.0,3.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


<a id='drawing'></a>
# Drawing spatialization


<a id='utilitary'></a>
## Utilitary functions
* Recoding lots of basic functions so they work with numba


### SQL-join between two 2-columns arrays
* [back to summary](#summary)

In [9]:
@njit
def npjoin(a_keycol, a, b_keycol, b, outer_value):
    """
    a_keycol, b_keycol, a, b : all are np.array
    Interpreted as 2 arrays of 2 columns
    sql-style outer join np.arrays a and b on columns akey and bkey
    assume a is sorted by akey and b by bkey
    assume keys do not have repeating values
    Outer join but missing keys are not assigned NaN, rather a (small) value
    Because we may have no key in common
    also it smoothes a bit the impossible flows
    return np of 3 columns :
        key, aval, bval
    """
    r = np.empty((a.shape[0]+b.shape[0], 3))
    ia = 0
    ib = 0
    ir = 0
    while ia<len(a) and ib<len(b):
        if a_keycol[ia]<b_keycol[ib]:
            r[ir, 0] = a_keycol[ia]
            r[ir, 1] = a[ia]
            r[ir, 2] = outer_value
            ia += 1
            ir += 1
            
        elif a_keycol[ia]>b_keycol[ib]:
            r[ir, 0] = b_keycol[ib]
            r[ir, 1] = outer_value
            r[ir, 2] = b[ib]
            ib += 1
            ir += 1

        else:
            r[ir, 0] = a_keycol[ia]
            r[ir, 1] = a[ia]
            r[ir, 2] = b[ib]
            ir += 1
            ia += 1
            ib += 1
    
    while ia<len(a):
        r[ir, 0] = a_keycol[ia]
        r[ir, 1] = a[ia]
        r[ir, 2] = outer_value
        ia += 1
        ir += 1
    
    while ib<len(b):
        r[ir, 0] = b_keycol[ib]
        r[ir, 1] = outer_value
        r[ir, 2] = b[ib]
        ib += 1
        ir += 1

    return r[:ir]

# import timeit
# n = 10000
# m = int(1.5*n)
# akey = np.random.choice(m, n)
# sortind = np.argsort(akey)
# a = np.arange(n)[sortind]
# akey = akey[sortind]
# 
# bkey = np.random.choice(m, n)
# sortind = np.argsort(bkey)
# b = np.arange(n)[sortind]
# bkey = bkey[sortind]
# 
# npjoin(akey, a , bkey, b, 0.01)
# %timeit npjoin(akey, a , bkey, b, 0.01)

npjoin(np.array([1,3]), 
       np.array([10,30]),
       np.array([1,2,4]), 
       np.array([100, 200,400]), 0)

npjoin(np.zeros(shape=0), 
       np.zeros(shape=0),
       np.array([1,2,4]), 
       np.array([100, 200,400]), 0)


array([[  1.,   0., 100.],
       [  2.,   0., 200.],
       [  4.,   0., 400.]])

#### Wrapper for `npjoin` that return output in the same foramt as input

In [10]:
@njit
def join_odz(o_list, proba_o, d_list, proba_d, eps=1e-10):
    """joining proba of o with proba of d"""
    z_arr = npjoin(o_list, proba_o, d_list, proba_d, eps)
    z_list = z_arr[:,0]
    
    new_probao_col = 1
    new_probad_col = 2
    proba_z = z_arr[:, new_probao_col] * z_arr[:, new_probad_col]
    return z_list, proba_z

### tile
* re-implement np.tile because it's not managed by numba
* [back to summary](#summary)

In [11]:
@njit
def nptile(a, n):
    """
    re-implement np.tile because it's not managed by numba
    """
    return a.repeat(n).reshape((-1, n)).T.flatten()

nptile(np.array([1,2,3]), 2)

print('custom nptile:')
%timeit nptile(np.array([1,2,3]), 5)
print('original nptile:')
%timeit np.tile(np.array([1,2,3]), 5)

custom nptile:
2.32 µs ± 93.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
original nptile:
5.52 µs ± 89.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### choice
* re-implement np.random.choice because weighting is not managed by numba.
* [back to summary](#summary)

In [12]:
@njit
def npchoice(a, p):
    """
    re-implement np.random.choice because weighting is not managed by numba.
    Is 2x faster if we don't have to cumsum here, but for our case it's cleaner to put the cumsum here
    and not slower because we never sample twice with the same p
    /!\ Unlike np.choice:
        - sample a unique element of a with probability array p
    """
    if len(a)==0:
        raise ValueError('given array is empty !')
    r = np.random.rand()
    return a[np.searchsorted(p.cumsum(), r, side='left')]

a = np.arange(500)
p = np.random.rand(len(a))
p = p/p.sum()
npchoice(a, p)
print('custom npchoice:')
%timeit npchoice(a, p)
print('original np.random.choice:')
%timeit np.random.choice(a, p=p)


custom npchoice:
1.37 µs ± 19.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
original np.random.choice:
25.2 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<a id='parameters'></a>
## Spatialisation parameters
* [back to summary](#summary)

In [13]:
# Nb of iterations to warm MCMC before sampling, w.r.t. autonomy
nb_warm_iter_dict = 1000 * np.ones(10, dtype=int)
nb_warm_iter_dict[1] = 1
nb_warm_iter_dict[2] = 50
nb_warm_iter_dict[3] = 200

# Nb of iterations we discard between 2 samples of MCMC w.r.t. autonomy
nb_cooldown_iter_dict = 1000 * np.ones(10, dtype=int)
nb_cooldown_iter_dict[1] = 0
nb_cooldown_iter_dict[2] = 10
nb_cooldown_iter_dict[3] = 50


<a id='preproc'></a>
## Preparing df to dicts of arrays
* we transform all pd.DataFrame to np.array, 
* [back to summary](#summary)

### making sure we find the right columns
* <font color='red'>**v8:** no column for mode </font>
* [back to summary](#summary)

In [14]:

#trans_matrix_arr = trans_matrix[['o', 'd', 't', 'proba_d']].values
trans_matrix_arr = trans_matrix[['o', 'd', 't', 'vol']].values
o_col, d_col, t_col, probad_col = 0, 1, 2, 3

#assert (trans_matrix['proba_d'] == trans_matrix_arr[:,probad_col]).all()
assert (trans_matrix['t'] == trans_matrix_arr[:,t_col]).all()
assert (trans_matrix['d'] == trans_matrix_arr[:,d_col]).all()
assert (trans_matrix['o'] == trans_matrix_arr[:,o_col]).all()

iris_commune_arr = iris_commune.values
z_col, neighbor_0_col = 0, 5
assert (iris_commune['iris_or_commune'] == iris_commune_arr[:, z_col]).all()
assert (iris_commune['nearest_neighbor_0'] == iris_commune_arr[:, neighbor_0_col]).all()
max_neighbor_rank = 20

motif_col_names = ['dep_{}_motif'.format(i) for i in range(max_chain_len+1)]
motif_cols = np.arange(len(motif_col_names), dtype=int)
time_col_names = ['dep_{}_time'.format(i) for i in range(1, max_chain_len+1)]
time_cols = np.arange(len(motif_col_names), len(motif_col_names)+len(time_col_names), dtype=int)

chain_len_col = len(motif_cols) + len(time_cols)
home_col = chain_len_col + 1
vol_col = home_col + 1
autonomy_col = vol_col + 1

### ztr_to_i
* we slice our arrays into numba dicts but tuples are not accepted as keys in numba dicts

* [back to summary](#summary)

In [15]:
@njit
def zt_to_i(z,t):
    """tuples (z,t) are not accepted as keys in numba dicts"""
    return z*1000+t

### `create_trans_matrix_d_dict_numba`
* transforming the transition matrix to dicts:
* `d_list_dict`: dict of (origin, timestep, transport mode): list of each possible destination
* `proba_d_dict`: dict of (origin, timestep, transport mode): list of probas for each possible destination
* <font color='red'>**v8:** no mode</font>
* [back to summary](#summary)

In [16]:
timesteps = trans_matrix['t'].unique()
from numba.typed import Dict
from numba.core import types

float_array = types.float64[:]
float_type = types.float64

def create_trans_matrix_d_dict_numba(trans_matrix_arr):
    
    # Make dictionary
    d_list_dict = Dict.empty(
        key_type=float_type,
        value_type=float_array,
    )
    proba_d_dict = Dict.empty(
    key_type=float_type,
    value_type=float_array
    )
    
    for o in tqdm(np.unique(trans_matrix_arr[:,o_col])):
        trans_matrix_sub_o = trans_matrix_arr[(trans_matrix_arr[:, o_col]==o)]
        for t in timesteps:
            trans_matrix_sub = trans_matrix_sub_o[(trans_matrix_sub_o[:, t_col]==t)].astype(np.float64)

            k = zt_to_i(o,t)
            sort_ind = np.argsort(trans_matrix_sub[:,d_col])
            
            d_list_dict[k] = trans_matrix_sub[sort_ind, d_col]
            proba_d_dict[k] = trans_matrix_sub[sort_ind, probad_col]
        

    return d_list_dict, proba_d_dict


d_list_dict, proba_d_dict = create_trans_matrix_d_dict_numba(trans_matrix_arr)
    

100%|████████████████████████████████████████| 536/536 [00:03<00:00, 136.66it/s]


### `create_trans_matrix_o_dict_numba`
* transforming the transition matrix to dicts:
* `o_list_dict`: dict of (destination, timestep, transport mode): list of each possible origin
* `proba_o_dict`: dict of (destination, timestep, transport mode): list of probas for each possible origin
* <font color='red'>**v8:** no mode</font>
* [back to summary](#summary)

In [17]:

def create_trans_matrix_o_dict_numba(trans_matrix_arr):
    # numba-ficiation of this does makes the notebook stop responding
    
    # Make dictionary
    o_list_dict = Dict.empty(
        key_type=float_type,
        value_type=float_array,
    )
    proba_o_dict = Dict.empty(
    key_type=float_type,
    value_type=float_array
    )
    
    for d in tqdm(np.unique(trans_matrix_arr[:,d_col])):
        trans_matrix_sub_d = trans_matrix_arr[(trans_matrix_arr[:, d_col]==d)]
        for t in timesteps:
            trans_matrix_sub = trans_matrix_sub_d[(trans_matrix_sub_d[:, t_col]==t)]

            k = zt_to_i(d,t)
            sort_ind = np.argsort(trans_matrix_sub[:,o_col])
            
            o_list_dict[k] = trans_matrix_sub[sort_ind, o_col]
            proba_o_dict[k] = trans_matrix_sub[sort_ind, probad_col]
        
    return o_list_dict, proba_o_dict


o_list_dict, proba_o_dict = create_trans_matrix_o_dict_numba(trans_matrix_arr)
    

100%|████████████████████████████████████████| 536/536 [00:03<00:00, 150.78it/s]


<a id='proba'></a>
## Functions to query probas

### `get_proba_o` and `get_proba_d`
* <font color='red'>**v8:** no mode</font>
* [back to summary](#summary)

In [18]:

@njit
def get_proba_o(o_list_dict, proba_o_dict, d, t):
    """
    return proba of each o given d, t
    d: destination
    t: timestep
    
    If no origin exist for this dt, look into the nearest neighbor of d
    iteratively as long as we didn't exhaust the list of nearest neighbors.
    """
    key = zt_to_i(d,t)
    o_list = o_list_dict.get(key)
    
    # getting the transitions from nearest zone instead
    if o_list is None or len(o_list)==0:
        neighbor_col = neighbor_0_col
        z_row = iris_commune_arr[iris_commune_arr[:, z_col]==d]
        while (neighbor_col<z_row.shape[1]) and ((o_list is None) or (len(o_list)==0)):
            if neighbor_col==z_row.shape[1]:
                return np.empty(shape=0), np.empty(shape=0)

            nn = z_row[:, neighbor_col].item()
            key = zt_to_i(nn,t)
            o_list = o_list_dict.get(key)
            neighbor_col += 1
            
    proba_o = proba_o_dict[key]
    return o_list, proba_o

@njit
def get_proba_d(d_list_dict, proba_d_dict, o, t):
    """
    return proba of each d given o, t, m
    o: origin
    t: timestep
    
    If no destination exist for this otm, look into the nearest neighbor of o
    iteratively as long as we didn't exhaust the list of nearest neighbors.
    """
    key = zt_to_i(o,t)
    d_list = d_list_dict.get(key)
    
    # getting the transitions from nearest zone instead
    if d_list is None or len(d_list)==0:
        neighbor_col = neighbor_0_col
        z_row = iris_commune_arr[iris_commune_arr[:, z_col]==o]
        while (neighbor_col<z_row.shape[1]) and ((d_list is None) or (len(d_list)==0)):
            if neighbor_col==z_row.shape[1]:
                return np.empty(shape=0), np.empty(shape=0)
            nn = z_row[:, neighbor_col].item()
            key = zt_to_i(nn,t)
            d_list = d_list_dict.get(key)
            neighbor_col += 1

    proba_d = proba_d_dict[key]
    return d_list, proba_d


### `get_proba_z`
* is proportional to `proba_o * proba_d`
* [back to summary](#summary)

In [19]:
@njit
def get_proba_z(row, locations, o_list_dict, proba_o_dict, 
                           d_list_dict, proba_d_dict, 
                dim, time_cols):
    time_col = time_cols[dim]
    time_prev_col = time_cols[dim-1]
    
    if dim==0:  # first place may not be home
        z_list, proba_z = get_proba_o(o_list_dict, proba_o_dict,
                              locations[dim+1],  # destination
                              row[time_col], # curr timestep
                             )

    elif dim==len(locations)-1:  # last place may not be home
        z_list, proba_z = get_proba_d(d_list_dict, proba_d_dict,
                                  locations[dim-1], # origin
                                  row[time_prev_col], # prev timestep
                                 )
                
    else:
        o_list, proba_o = get_proba_o(o_list_dict, proba_o_dict,
                              locations[dim+1],  # destination
                              row[time_col], # curr timestep
                             )
        
        d_list, proba_d = get_proba_d(d_list_dict, proba_d_dict,
                              locations[dim-1], # origin
                              row[time_prev_col], # prev timestep
                             )

        z_list, proba_z = join_odz(o_list, proba_o, d_list, proba_d, eps=1e-10)

    return z_list, proba_z

In [20]:
@njit
def get_link_mask(purpose_row):
    """
    input : a list of activity purposes
    return : a list of ids for linksets. 
        Activities belonging to the same linkset are expected to have the same location.
    """
    linkset_ids = np.zeros(shape=len(purpose_row), dtype=np.int32)
    max_state_id = 0
    purpose_visited = -np.ones(len(purpose_row))  # stash list
    purpose_to_id = -np.ones(len(purpose_row))  # dict list
    for i, p in enumerate(purpose_row.astype(np.int32)):
        if p in purpose_visited:
            linkset_ids[i] = purpose_to_id[p]
        else:
            if (p>=0) and (p<=2):  # home, work or study
                purpose_visited[max_state_id] = p
                purpose_to_id[p] = max_state_id
                linkset_ids[i] = max_state_id
                max_state_id += 1
            else:
                linkset_ids[i] = max_state_id
                max_state_id += 1
    return linkset_ids

@njit
def get_non_singleton_linksets(link_mask):
    return np.array([i for i in np.unique(link_mask) if (link_mask==i).sum()>1])


purposes_example = np.array([0.,2.,2.,3.,4.,3.,0.])
print('activity purposes: {}'.format(purposes_example))
lm = get_link_mask(purposes_example)
print('link sets : {}'.format(lm))
print('activities appearing more than once: {}'.format(get_non_singleton_linksets(lm)))

activity purposes: [0. 2. 2. 3. 4. 3. 0.]
link sets : [0 1 1 2 3 4 0]
activities appearing more than once: [0 1]


In [21]:


@njit
def get_proba_z_linked(row, locations, o_list_dict, proba_o_dict, 
                           d_list_dict, proba_d_dict, 
                            dim, time_cols, chain_len):
    """
    get_proba_z but handles the linked states.
    /!\ Assumes linked states NEVER follow each others.
    """
    link_mask = get_link_mask(row[:chain_len])
    non_singleton_linksets = get_non_singleton_linksets(link_mask)
    
    
    if link_mask[dim] in non_singleton_linksets:
        #dict (neighboring linkset):(id of each possible zone for current linkset)
        list_linkset_dict = Dict.empty(key_type=types.int32, value_type=float_array)

        #dict (neighboring linkset):(proba of each possible zone for current linkset)
        proba_linkset_dict = Dict.empty(key_type=types.int32, value_type=float_array)
        
        # number of factors in common with each linkset
        exponant_linksets = np.zeros(link_mask.max()+1)

        for dim_linked in np.where(row[:chain_len]==row[dim])[0]:
            if dim_linked<chain_len-1:
                o_list_linked, proba_o_linked = get_proba_o(o_list_dict, proba_o_dict, 
                                                            locations[dim_linked+1],
                                                            row[time_cols[dim_linked]])

                next_purpose = link_mask[dim_linked+1]
                exponant_linksets[next_purpose] += 1
                if next_purpose in proba_linkset_dict:
                    res_list, proba_res = join_odz(list_linkset_dict[next_purpose], 
                                                   proba_linkset_dict[next_purpose],
                                                   o_list_linked, 
                                                   proba_o_linked)
                    list_linkset_dict[next_purpose] = res_list
                    proba_linkset_dict[next_purpose] = proba_res
                else:
                    list_linkset_dict[next_purpose] = o_list_linked
                    proba_linkset_dict[next_purpose] = proba_o_linked
                    
            if dim_linked>0:
                d_list_linked, proba_d_linked = get_proba_d(d_list_dict, proba_d_dict, 
                                                            locations[dim_linked-1], 
                                                            row[time_cols[dim_linked-1]])

                prev_purpose = link_mask[dim_linked-1]
                exponant_linksets[prev_purpose] += 1
                if prev_purpose in proba_linkset_dict:
                    res_list, proba_res = join_odz(list_linkset_dict[prev_purpose], 
                                                   proba_linkset_dict[prev_purpose],
                                                   o_list_linked, 
                                                   proba_o_linked)
                    list_linkset_dict[prev_purpose] = res_list
                    proba_linkset_dict[prev_purpose] = proba_res
                else:
                    list_linkset_dict[prev_purpose] = d_list_linked
                    proba_linkset_dict[prev_purpose] = proba_d_linked

                    
            # count double transitions that we see in factor graphs
            # (a bit optional, doesn't change much the result anyway)
            if (dim_linked+2<len(link_mask)) and (link_mask[dim_linked+2]==link_mask[dim_linked]):
                if dim_linked>0 and (link_mask[dim_linked-1]==link_mask[dim_linked+1]):
                    exponant_linksets[next_purpose] += 1
                    res_list, proba_res = join_odz(list_linkset_dict[next_purpose], 
                                                   proba_linkset_dict[next_purpose],
                                                   o_list_linked, 
                                                   proba_o_linked)
                    list_linkset_dict[next_purpose] = res_list
                    proba_linkset_dict[next_purpose] = proba_res
                    
            if (dim_linked-2>=0) and (link_mask[dim_linked-2]==link_mask[dim_linked]):
                if dim_linked+1<len(link_mask) and (link_mask[dim_linked-1]==link_mask[dim_linked+1]):
                    exponant_linksets[prev_purpose] += 1
                    res_list, proba_res = join_odz(list_linkset_dict[prev_purpose], 
                                                   proba_linkset_dict[prev_purpose],
                                                   o_list_linked, 
                                                   proba_o_linked)
                    list_linkset_dict[prev_purpose] = res_list
                    proba_linkset_dict[prev_purpose] = proba_res
            #####
            
        # Applying root corrections and multiplying all proba laws
        z_list, proba_z = np.zeros(shape=0), np.zeros(shape=0)
        for i in range(len(exponant_linksets)):
            if exponant_linksets[i]>0:
                proba_linkset_dict[i] = np.power(proba_linkset_dict[i], 1/exponant_linksets[i])
                z_list, proba_z = join_odz(list_linkset_dict[i], proba_linkset_dict[i], z_list, proba_z)

    else: # linkset is singleton
        z_list, proba_z = get_proba_z(row, locations, o_list_dict, proba_o_dict, 
                                                      d_list_dict, proba_d_dict, 
                                                      dim, time_cols)

        # if prev and next dims are in the same linkset:
        if dim>0 and dim<chain_len-1 and (row[dim-1]==row[dim+1]) and (row[dim-1]>=0) and (row[dim-1]<=2):
            proba_z = np.sqrt(proba_z)

    return z_list, proba_z

<a id='running'></a>
## Running spatialisation in itself
* [back to summary](#summary)

In [22]:
int_type = types.int8
int_array = types.int8[:]

#@njit(parallel=True)
def spatialize(synthpop_grouped_sub_arr, nb_warm_iter_dict, nb_cooldown_iter_dict, 
              o_list_dict, proba_o_dict, d_list_dict, proba_d_dict,
              progress_proxy):
    locations_sampled = np.zeros(shape=(int(synthpop_grouped_sub_arr[:, vol_col].sum()), max_chain_len+1))
    cum_vols = synthpop_grouped_sub_arr[:, vol_col].cumsum()
    
    for irow in prange(len(synthpop_grouped_sub_arr)):
        row = synthpop_grouped_sub_arr[irow]
        
        # Getting max autonomy of segments of the agenda
        autonomy = int(row[autonomy_col])
        nb_iter = int(nb_warm_iter_dict[autonomy]+row[vol_col]*(nb_cooldown_iter_dict[autonomy] + 1))
    
        chain_len = int(row[chain_len_col]+1)

        locations = np.random.choice(a=iris_commune_arr[:, z_col], size=(chain_len))
        
        # first cols are about motive
        home_mask = (row[:chain_len]==0)

        # Something smart to not have to browse the entire agenda each time we want to fetch linked states:
        #link_dict = Dict.empty(key_type=int_type, value_type=int_array)
        #for i, motive in enumerate(row[:chain_len]):
        #    if i not in link_dict:
        #        if motive==1 or motive==2:
        #            link_dict[i] = np.where(row[:chain_len]==motive)[0].astype(np.int8)
        #        elif motive != 0:
        #            link_dict[i] = i
        
        locations[home_mask] = row[home_col]
        draw_mask = ~home_mask

        draw_indices = draw_mask.nonzero()[0] # indices we will have to draw in MCMC
        
        # # pre-fetching phase, not implemented
        # for i in draw_indices:
        #     ...
        
        # Sampling each dimension in order, this way we make sure all segments are drawn 
        # One iteration is drawing all states that have to be drawn, in order.
        # We could take a random order but then we wouldn't be sure that all states have been drawn
        # in long segments for chains that have a long segment and a short one.
        dim_to_draw = nptile(draw_indices, nb_iter)

        # i_sample: index of where we save the trajectory we draw in locations_sampled.
        # Each trajectory can concern several agents, so we may have to sample it more than once.
        # So each trajectory has an assigned slot for the first sample, and the following samples
        # are saved in the follow rows.
        if irow==0:
            i_sample=0
        else:
            i_sample = int(cum_vols[irow-1])
        
        for i, dim in enumerate(dim_to_draw):
            z_list, proba_z = get_proba_z_linked(row, locations, o_list_dict, proba_o_dict, 
                                                  d_list_dict, proba_d_dict, 
                                                  dim, time_cols, chain_len)
            proba_z = proba_z/proba_z.sum()

            tirage = npchoice(z_list, proba_z)
            locations[dim] = tirage
            
            ## Sampling once as soon as we finished warmup, and once each time the cooldown is finished
            if ((i>=nb_warm_iter_dict[autonomy]*len(draw_indices)) and 
             (i-len(draw_indices)*nb_warm_iter_dict[autonomy]) % (len(draw_indices)*(nb_cooldown_iter_dict[autonomy] + 1)) == 0):
                locations_sampled[i_sample, :chain_len] = locations
                i_sample+=1
                
        progress_proxy.update(1)
    return locations_sampled

# 00:15:00
synthpop_grouped_sub = synthpop_grouped
synthpop_grouped_sub_arr = synthpop_grouped_sub[motif_col_names + 
                                                time_col_names+
                                                ['chain_len', 'iris_or_commune', 'vol', 'autonomy']].values
with ProgressBar(total=len(synthpop_grouped_sub_arr)) as progress:
    locations_sampled = spatialize(synthpop_grouped_sub_arr, nb_warm_iter_dict, nb_cooldown_iter_dict,
                                   o_list_dict, proba_o_dict, d_list_dict, proba_d_dict,
                                   progress)

  0%|                                                | 0/390221 [00:00<?, ?it/s]

/var/folders/4m/96kct6fd1vl7ty8sc_mpj7kw0000gp/T/ipykernel_26036/1775577505.py:88: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  proba_linkset_dict[i] = np.power(proba_linkset_dict[i], 1/exponant_linksets[i])


<a id='sanity'></a>
# Sanity checks
* [back to summary](#summary)

* No trajectory has failed to be attributed something

In [23]:
print('Nb of trajectories that are only 0\'s: {}'.format(np.all(locations_sampled==0,axis=1).sum()))

Nb of trajectories that are only 0's: 0


* Right amount of chains of each len. 
* **Note:** Offset of 1 in chain_len is explained because `locations_sampled` also has a column for the location 0.

In [24]:
print('Distribution of chain_len in synthpop:\n')
print(synthpop.groupby('chain_len').size())
print('\nDistribution of len of location chains sampled:')
chain_len_locations = np.sum((locations_sampled!=0),axis=1)
print(pd.DataFrame({'nb_locations':chain_len_locations}).groupby('nb_locations').size())

Distribution of chain_len in synthpop:

chain_len
0     283041
1      21920
2     555982
3     105410
4     260108
5      55532
6      40492
7       9446
8       2962
9        493
10       400
dtype: int64

Distribution of len of location chains sampled:
nb_locations
2      21920
3     555982
4     105410
5     260108
6      55532
7      40492
8       9446
9       2962
10       493
11       400
dtype: int64


* But the order is wrong !
* Order seems consistent with the order of `synthpop_grouped`
* So it's the groupby that messed up our order

In [25]:
print('Fraction of consistent chain len between synthpop_grouped and location chains sampled:')
print('{:.0f}%'.format(100*(np.repeat(synthpop_grouped['chain_len'], 
                                      synthpop_grouped['vol'])==chain_len_locations-1).mean()))

Fraction of consistent chain len between synthpop_grouped and location chains sampled:
100%


* Joining agendas with an ad hoc key

In [26]:
# 00:00:02

id_agent = (synthpop_grouped['id_agent'].explode()).values

location_df = pd.DataFrame(locations_sampled, columns=['dep_{}_zone'.format(i) for i in range(max_chain_len+1)])
location_df['id_agent'] = id_agent

synthpop_sampled = (synthpop
                    .drop(columns=['dep_{}_zone'.format(i) for i in range(max_chain_len+1)])
                    .merge(location_df, on='id_agent'))

* Check that we have no doublon or disappearing agent in the merge:

In [27]:
print('len of synthpop: \t {}'.format(len(synthpop)))
print('len of location chains:  {}'.format(len(locations_sampled)))
print('len of synthpop merged:  {}'.format(len(synthpop_sampled)))

len of synthpop: 	 1335786
len of location chains:  1052745
len of synthpop merged:  1052745


* Static trajectories

In [28]:
zone_cols = ['dep_{}_zone'.format(i) for i in range(max_chain_len+1)]
synthpop[zone_cols] = synthpop[zone_cols].fillna(0)
synthpop_locations_sampled = synthpop[zone_cols].values
moved = (synthpop_locations_sampled[:,1:] != synthpop_locations_sampled[:,:-1])
padding = synthpop_locations_sampled[:,1:]==0
static_traj = (~(moved & ~padding)).all(axis=1)
print('{:.2f}% of agents don\'t move at all'.format(100*static_traj.mean()))

28.54% of agents don't move at all


In [29]:
(synthpop_sampled['chain_len']==0).mean()

0.0

<a id='add_empty'></a>
# Adding empty agendas
* They have been removed from the dataset at the beginning
* Now put them back
* [back to summary](#summary)

In [30]:
for i in range(max_chain_len+1):
    synthpop_empty_agenda['dep_{}_zone'.format(i)] = 0
synthpop_sampled = pd.concat([synthpop_sampled, synthpop_empty_agenda])

<a id='export'></a>
# Export
* [back to summary](#summary)

In [31]:
outpath = synthpop_path[:-4] + '_spatializev8h2.csv'
synthpop_sampled.to_csv(outpath, index=False)

print(datetime.datetime.now())
print(outpath)
synthpop_sampled.head()

2023-12-08 18:13:18.646673
/Users/benoit/Desktop/Pro/210526-fusion/outdata/synthpop/synthpop_statmatch_rescaleipu_spatializev8h2.csv


,iris_or_commune,gender,has_car,occupation,age,home_status,main_transport_work,id_agenda,dep_0_motif,outsider,...,dep_1_zone,dep_2_zone,dep_3_zone,dep_4_zone,dep_5_zone,dep_6_zone,dep_7_zone,dep_8_zone,dep_9_zone,dep_10_zone
0,690270102,0,1,8,3,0,-1,1320021221,0.0,0.0,...,691000101.0,690270102.0,692020401.0,69116.0,690270102.0,0.0,0.0,0.0,0.0,0.0
1,69288,0,1,6,1,0,2,1320021221,0.0,0.0,...,692870101.0,69288.0,69289.0,692870102.0,69288.0,0.0,0.0,0.0,0.0,0.0
2,69298,0,1,6,2,0,5,1320021221,0.0,0.0,...,692590602.0,69298.0,69289.0,69289.0,69298.0,0.0,0.0,0.0,0.0,0.0
3,692661301,0,1,7,0,0,-1,1320021221,0.0,0.0,...,693850201.0,692661301.0,693830301.0,693830301.0,692661301.0,0.0,0.0,0.0,0.0,0.0
4,690890301,0,1,5,2,0,-1,1320021221,0.0,0.0,...,692440201.0,690890301.0,69044.0,69044.0,690890301.0,0.0,0.0,0.0,0.0,0.0
